In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from random import shuffle
from IPython.display import clear_output
from sklearn import metrics
from tqdm import tqdm
import time
import seaborn as sns
%matplotlib inline

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import os
USE_GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '3' if USE_GPU else ''
EPS = 1e-15

In [4]:
import ROOT
import rootpy
import root_numpy
import uproot

/opt/conda/anaconda3/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.15.0 is currently installed but you installed root_numpy against numpy 1.12.1. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


## Convert .root to .csv

In [5]:
r = uproot.open("../data/hits.root")
r.keys()

[b'lpmt_hits;187',
 b'lpmt_hits;186',
 b'spmt_hits;8',
 b'spmt_hits;7',
 b'true_info;1',
 b'lpmt_pos;1',
 b'spmt_pos;1']

In [6]:
t1 = r.get("lpmt_hits")
t2 = r.get("spmt_hits")
t3 = r.get("true_info")
t4 = r.get("lpmt_pos")
t5 = r.get("spmt_pos")

In [7]:
from utils.converter import convert

In [8]:
convert(t1, t2, t3, t4, t5)

100%|██████████| 100000/100000 [01:10<00:00, 1422.68it/s]


## Data description

lpmt — большие ФЭУ, spmt — маленькие
hits — зарегистрированные фото-электроны (не все фотоны выбивают фото-электрон, и не всякий фото-электрон регистрируется)
Каждый hit имеет 3 характеристики: номер ФЭУ (pmtID), время от физического события в детекторе (hitTime) и происхождение (isDN=0 для нормальных хитов, isDN=1 для шума в самих ФЭУ).

В каждом физическом событии может быть переменное количество хитов. Количество хитов сохранено в переменной nHits. Технически nHits — это длина массивов pmtID, hitTime и isDN в каждом событии.

В true_info записаны данные, которые нельзя напрямую измерить, но можно предсказывать: E — енергия в МэВ; x,y,z,R — координаты в мм (R^2 = x^2+y^2+z^2). События в деревьях соответствуют друг другу по порядковому номеру: певрое из true_info соответствует первому из lpmt_hits и первому из spmt_hits.

lpmt_pos и spmt_pos — это координаты ФЭУ (x, y, z в миллиметрах).

## Exploratory Data Analysis

In [10]:
lpmt_hits = pd.read_hdf('../data/lpmt_hits.h5', mode='r') 
lpmt_hits.shape

(929768817, 4)

In [11]:
lpmt_hits.head()

,event,hitTime,isDN,pmtID
0,0,249.992615,False,14175
1,0,40.010311,False,17319
2,0,162.123199,False,16882
3,0,51.875614,False,14951
4,0,79.817497,False,10947


In [12]:
spmt_hits = pd.read_hdf('../data/spmt_hits.h5', mode='r')
spmt_hits.shape

(33958478, 4)

In [13]:
spmt_hits.head()

,event,hitTime,isDN,pmtID
0,0,37.040001,False,318069
1,0,238.278091,False,311819
2,0,103.316055,False,307244
3,0,108.034225,False,323230
4,0,36.505154,False,320568


In [15]:
true_info = pd.read_csv('true_info.csv') 
true_info.shape

(100000, 6)

In [16]:
true_info.head()

,E,R,evtID,x,y,z
0,4.747791,14610.378,0,8290.7790,11995.6180,911.74286
1,3.919721,14630.141,1,11397.6320,5407.4497,-7409.08200
2,6.823932,14573.132,2,14063.3380,-3812.8540,246.65280
3,3.765940,16820.080,3,-2377.9307,-16317.7020,3315.59030
4,3.217473,13026.938,4,-8617.1170,868.5116,9730.98600


In [17]:
lpmt_pos = pd.read_csv('lpmt_pos.csv') 
lpmt_pos.shape

(17739, 4)

In [18]:
lpmt_pos.head()

,pmt_id,pmt_x,pmt_y,pmt_z
0,0,1.049021e+03,0.0000,19171.32
1,1,9.084785e+02,524.5103,19171.32
2,2,5.245103e+02,908.4785,19171.32
3,3,6.423399e-14,1049.0206,19171.32
4,4,-5.245103e+02,908.4785,19171.32


In [19]:
spmt_pos = pd.read_csv('spmt_pos.csv') 
spmt_pos.shape

(24952, 4)

In [20]:
spmt_pos.head()

,pmt_id,pmt_x,pmt_y,pmt_z
0,300000,1.384810e+03,244.17944,19148.438
1,300001,1.217782e+03,703.08670,19148.438
2,300002,9.038709e+02,1077.19130,19148.438
3,300003,4.809396e+02,1321.37070,19148.438
4,300004,8.610328e-14,1406.17330,19148.438
